In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import warnings

class ETendersScraper:
    def __init__(self, driver_path):
        self.driver = webdriver.Chrome(executable_path=driver_path)
        self.base_url = 'https://etenders.gov.in/eprocure/app'

    def navigate_to_tenders_page(self):
        self.driver.get(self.base_url)
        tenders = self.driver.find_element(By.XPATH, "/html/body/table/tbody/tr[1]/td/table/tbody/tr[4]/td/table/tbody/tr/td[2]/span/span[2]/span[1]/a")
        tenders.click()



    def scrape_tender_data(self):
        e_published_date = []
        bid_closing_date = []
        tender_opening_date = []
        title = []
        organisation = []
        value = []

        start = 0
        end = 250

        for page in range(start, end):
            try:
                captcha = input("Enter Captcha: ")
                
                enter = self.driver.find_element(By.XPATH, '//*[@id="captchaText"]')
                enter.send_keys(captcha)

                search = self.driver.find_element(By.XPATH, "/html/body/div/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td/table/tbody/tr[7]/td/input")
                search.click()
                
                time.sleep(3)
            
            except NoSuchElementException as e:
                print ("There was an error restart scraping")
                break 
          


            try:
                p_date = self.driver.find_elements(By.XPATH, '//*[@class="even" or @class="odd"]/td[2]')
                for i in p_date:
                    p = i.text
                    e_published_date.append(p)
            except NoSuchElementException as e:
                e_published_date.append('-')

            try:
                b_close_date = self.driver.find_elements(By.XPATH, '//*[@class="even" or @class="odd"]/td[3]')
                for i in b_close_date:
                    b = i.text
                    bid_closing_date.append(b)
            except NoSuchElementException as e:
                bid_closing_date.append('-')

            try:
                tender_open_date = self.driver.find_elements(By.XPATH, '//*[@class="even" or @class="odd"]/td[4]')
                for i in tender_open_date:
                    o = i.text
                    tender_opening_date.append(o)
            except NoSuchElementException as e:
                tender_opening_date.append('-')

            try:
                t = self.driver.find_elements(By.XPATH, '//*[@class="even" or @class="odd"]/td[5]')
                for i in t:
                    title.append(i.text)
            except NoSuchElementException as e:
                title.append('-')

            try:
                org = self.driver.find_elements(By.XPATH, '//*[@class="even" or @class="odd"]/td[6]')
                for i in org:
                    g = i.text
                    organisation.append(g)
            except NoSuchElementException as e:
                organisation.append('-')

            try:
                val = self.driver.find_elements(By.XPATH, '//*[@class="even" or @class="odd"]/td[7]')
                for i in val:
                    v = i.text
                    value.append(v)
            except NoSuchElementException as e:
                value.append('-')

            try:
                next_page = self.driver.find_element(By.ID, "linkFwd")
                next_page.click()
            except NoSuchElementException as e:
                print("Scraping Completed")
                break

        print(len(e_published_date), len(bid_closing_date), len(tender_opening_date), len(title), len(organisation), len(value))

        df = pd.DataFrame({'Published_date': e_published_date, 'Bid_closing_date': bid_closing_date,
                           'Tender_opening_date': tender_opening_date, 'Title': title, 'Organisation': organisation,
                           'Value': value})
        df.to_csv('tender.csv', index=False)

    def close_driver(self):
        self.driver.close()

if __name__ == "__main__":
    warnings.filterwarnings('ignore')
    driver_path = r"C:\Users\pooja\chromedriver-win32\chromedriver-win32\chromedriver.exe"
    scraper = ETendersScraper(driver_path)
    scraper.navigate_to_tenders_page()
    scraper.scrape_tender_data()
    scraper.close_driver()
